# SQL Alchemy

Szymon Dziubak

In [1]:
#Tutaj pokazane jest budowanie kodu krok po kroku
#w folderze utowrzone są pliki .py które pokazują działanie modułów

In [2]:
from sqlalchemy.orm import declarative_base #klasa bazowa do tworzenia struktur bazy
from sqlalchemy import Column, String, DateTime, Integer #pobieramy funkcje umożliwiające tworzenie kolumny i typów danych
from datetime import datetime 
from sqlalchemy.orm import sessionmaker 

In [3]:
#tworzymy klasę bazową
Base = declarative_base()

In [4]:
"""
W schemacie base tworzymy następujący obiekt:
class User 
    id int
    username str
    email str
    date_created datetime
"""

class User(Base): #dziedziczy z klasy Base do tworzenia obiektów
    __tablename__='users' #nazwa tabeli
    id = Column(Integer(), primary_key = True) #atrybut - klucz pierwotny
    username = Column(String(25), nullable = False, unique = True) #kolumna string niepusta i unikalna
    email = Column(String(80), unique = True, nullable = False) #kolumna string niepusta i unikalna
    date_created = Column(DateTime(),default = datetime.utcnow) #data i czas, domyślnie utcnow()

    def __repr__(self): #tekstowa reprezentacja klasy, dostaniemy dzięki temu wyświetlenie użytkownika
        return f"<User {self.username} email {self.email}>"

In [5]:
new_user = User(id = 1, username = "Szymon", email = "sd76805@student.sgh.waw.pl") #tworzymy nowego użytkownika jako egzemplarz klasy User
print(new_user)

<User Szymon email sd76805@student.sgh.waw.pl>


In [6]:
from sqlalchemy import create_engine 
import os

In [7]:
BASE_DIR = r"C:\Users\BPOL_1637\Documents\sgh\python\sqlalchemy"
#BASE_DIR = os.path.dirname(os.path.realpath(__file__)) #ścieżka obecnej lokalizacji

connection_string="sqlite:///"+os.path.join(BASE_DIR, 'site.db') # dane do otworzenia naszej bazy danych

#tworzy silnik bazy danych
engine = create_engine(connection_string, echo = True) #echo pokazuje komendy sql


In [8]:
#tworzenie silnika bazy danych

In [9]:
Base.metadata.create_all(engine) #tworzenie bazy danych
#polecenie wywietli kod SQL

2022-10-24 18:05:27,034 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-24 18:05:27,034 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-10-24 18:05:27,035 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-24 18:05:27,036 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2022-10-24 18:05:27,036 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-24 18:05:27,037 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	username VARCHAR(25) NOT NULL, 
	email VARCHAR(80) NOT NULL, 
	date_created DATETIME, 
	PRIMARY KEY (id), 
	UNIQUE (username), 
	UNIQUE (email)
)


2022-10-24 18:05:27,038 INFO sqlalchemy.engine.Engine [no key 0.00054s] ()
2022-10-24 18:05:27,045 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
#dodawanie danych

Session = sessionmaker()

local_session = Session(bind = engine) #tworzenie sesji użytkownika

new_user = User(username = "Jan", email = "Jan@Kowalski.pl") #definicja rekordu do bazy danych

local_session.add(new_user) #dodanie nowego rekordu do bazy danych
local_session.commit() #potwierdzenie operacji


#dodanie wielu użytkowników
users = [{
        "username":"Anna","email" : "anna@sgh.waw.pl"
    },{
        "username":"Kasia","email" : "kasia@sgh.waw.pl"
    },{
        "username":"Asia","email" : "asia@sgh.waw.pl"
    },{
        "username":"Adam","email" : "adam@sgh.waw.pl"

}]

for u in users:
    new_user = User(username = u['username'], email = u['email'])
    print(new_user)
    local_session.add(new_user)
    local_session.commit()
    print(f"Added {u['username']}")

2022-10-24 18:05:27,066 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-24 18:05:27,068 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-10-24 18:05:27,069 INFO sqlalchemy.engine.Engine [generated in 0.00106s] ('Jan', 'Jan@Kowalski.pl', '2022-10-24 16:05:27.068261')
2022-10-24 18:05:27,072 INFO sqlalchemy.engine.Engine COMMIT
<User Anna email anna@sgh.waw.pl>
2022-10-24 18:05:27,079 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-24 18:05:27,080 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, date_created) VALUES (?, ?, ?)
2022-10-24 18:05:27,081 INFO sqlalchemy.engine.Engine [cached since 0.01369s ago] ('Anna', 'anna@sgh.waw.pl', '2022-10-24 16:05:27.080779')
2022-10-24 18:05:27,084 INFO sqlalchemy.engine.Engine COMMIT
Added Anna
<User Kasia email kasia@sgh.waw.pl>
2022-10-24 18:05:27,089 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-24 18:05:27,091 INFO sqlalchemy.engine.Engine INSERT INTO u

In [11]:
#czytanie danych

In [12]:
local_session = Session(bind=engine)

users = local_session.query(User).all()[:3]

for user in users:
    print(user.username)

#query z filtrem
jan = local_session.query(User).filter(User.username == 'Jan').first()
print(jan)

2022-10-24 18:05:27,157 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-24 18:05:27,159 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users
2022-10-24 18:05:27,159 INFO sqlalchemy.engine.Engine [generated in 0.00063s] ()
Jan
Anna
Kasia
2022-10-24 18:05:27,163 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users 
WHERE users.username = ?
 LIMIT ? OFFSET ?
2022-10-24 18:05:27,163 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ('Jan', 1, 0)
<User Jan email Jan@Kowalski.pl>


In [13]:
#zmienianie danych

In [14]:
local_session=Session(bind=engine)

user_to_update = local_session.query(User).filter(User.username=="Jan").first()

user_to_update.username = "Andrzej"
user_to_update.email = "Andrzej@Kowalski.pl"

local_session.commit()

2022-10-24 18:05:27,189 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-24 18:05:27,190 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users 
WHERE users.username = ?
 LIMIT ? OFFSET ?
2022-10-24 18:05:27,191 INFO sqlalchemy.engine.Engine [cached since 0.02786s ago] ('Jan', 1, 0)
2022-10-24 18:05:27,193 INFO sqlalchemy.engine.Engine UPDATE users SET username=?, email=? WHERE users.id = ?
2022-10-24 18:05:27,193 INFO sqlalchemy.engine.Engine [generated in 0.00048s] ('Andrzej', 'Andrzej@Kowalski.pl', 1)
2022-10-24 18:05:27,195 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
#usuwanie danych

In [16]:
local_session = Session(bind=engine)

user_to_delete = local_session.query(User).filter(User.username == "Adam").first()

local_session.delete(user_to_delete)

local_session.commit()

2022-10-24 18:05:27,235 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-24 18:05:27,236 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users 
WHERE users.username = ?
 LIMIT ? OFFSET ?
2022-10-24 18:05:27,236 INFO sqlalchemy.engine.Engine [cached since 0.07346s ago] ('Adam', 1, 0)
2022-10-24 18:05:27,238 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2022-10-24 18:05:27,238 INFO sqlalchemy.engine.Engine [generated in 0.00047s] (5,)
2022-10-24 18:05:27,240 INFO sqlalchemy.engine.Engine COMMIT


In [17]:
#sortowanie danych

In [18]:
local_session = Session(bind=engine)

users = local_session.query(User).order_by(User.username.asc()).all()
print("Ascending:")
for user in users:
    print(user.username)


users_desc = local_session.query(User).order_by(User.username.desc()).all()
print("Descending:")
for user in users_desc:
    print(user.username)


2022-10-24 18:05:27,267 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-24 18:05:27,268 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users ORDER BY users.username ASC
2022-10-24 18:05:27,268 INFO sqlalchemy.engine.Engine [generated in 0.00051s] ()
Ascending:
Andrzej
Anna
Asia
Kasia
2022-10-24 18:05:27,270 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.date_created AS users_date_created 
FROM users ORDER BY users.username DESC
2022-10-24 18:05:27,271 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ()
Descending:
Kasia
Asia
Anna
Andrzej
